# Path 1 - Gemini API
This path will guide you on the set-up and usage of Google Gemini's API.

### 0. Get and store the API key
First of all, you need to login with your google account and get an API key [here](https://aistudio.google.com/app/apikey). It is **very important** that you do not share your API key with anyone and that you do not have it in your Repository.

You can keep your API key in a secure local document and access it when needed. It is common to save the key as an environmental variable so that it can be accessed by your python script.

However, this means your API key is in plain text in your script. To avoid this, if you're using VS Code, you can add your API key to a `.env` file in your workspace root with the following line:

```sh
GEMINI_API_KEY="PASTE YOUR KEY HERE"
```

Alternatively, you can use the [dot-env library](https://github.com/theskumar/python-dotenv).

In [53]:
# You can check if the environment variable API_KEY has been set up properly by running this line
from dotenv import load_dotenv

load_dotenv()

!if [ -z $GEMINI_API_KEY ]; then echo "\$GEMINI_API_KEY not found"; else echo "\$GEMINI_API_KEY found"; fi

$GEMINI_API_KEY found


I0000 00:00:1758263840.616645 5965093 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


### 1. First simple request
Now, you can write a simple script to see if everything is working properly.

In [54]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()  # here you can also pass the api_key directly using os.environ['GEMINI_API_KEY']
config = None

default_model = "gemini-2.5-flash"

client

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


#### Exercise 1

Ask the model to generate content about a random topic and print the response in text.

Here is the [official documentation](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#configure) to find the help you need.

In [55]:
# Your code here
from google.genai import types

response = client.models.generate_content(
    model=default_model,
    contents="Who are you?",
    config={} if config is None else config
)
print(response.text)

I am a large language model, trained by Google.


### 2. Generation parameters

When asking the model to generate some text, there are different parameters that you can tune to improve on the final quality of the text. [Here](https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters) is an overview of the parameters that Gemini offers. Try some of them in different context and understand how they affect the final generated text.

#### Exercise 2

Play with the output temperature, which controls the randomness of the generated text `temperature=0` means deterministic output, while `temperature=1` means maximum randomness (try some intermediate value too). Consider keeping the `max_output_tokens` to 50 so that the output is not too long; if you do, you should also set a low `thinking_budget` to avoid an empty response.

In [66]:
from google.genai import types

temp_vals = [0, 0.2, 0.4, 0.6, 0.8, 1]

for temp in temp_vals:
        config = types.GenerateContentConfig(
                temperature=temp
        )
        response = client.models.generate_content(
                model=default_model,
                contents="Who was the builder of the Turning Torso in Malmo, Sweden?",
                config=config
        )
        print(f'temp == {temp} --> {response.text}')

temp == 0 --> The Turning Torso was built by **HSB Malmö**, a cooperative housing association. They were the developer and commissioner of the project.

The building was designed by the Spanish architect, structural engineer, and sculptor **Santiago Calatrava**, who also served as the structural engineer.
temp == 0.2 --> The general contractor responsible for the construction of the Turning Torso was **NCC AB**.

However, it's also important to note:

*   **HSB Malmö** was the developer and client who commissioned the building.
*   **Santiago Calatrava** was the architect who designed it.
temp == 0.4 --> The **Turning Torso** was designed by Spanish architect **Santiago Calatrava**.

It was commissioned by **HSB Malmö**, a cooperative housing association, which was also the developer and original owner of the building.
temp == 0.6 --> The primary **developer and client** behind the Turning Torso was **HSB Malmö**, a Swedish housing cooperative.

The building was **designed** by the ren

#### Exercise 3

Try out different `top_k` values, which controls how many tokens the model considers for output `top_k=1` means the model considers only one token for output (the one with the highest probability) `top_k=50` means the model considers the top 50 tokens for output.

In [78]:
# Your code here
top_k_vals = [1,5,10,20,30,40,50]

for val in top_k_vals:
    config = types.GenerateContentConfig(
            temperature=0.4,
            top_p=0.95,
            top_k=val
            )
    response = client.models.generate_content(
                model=default_model,
                contents="Explain to me like I am 5: Why would a government grant subsidy to wheat farmers?",
                config=config
                )
    print(f'top_k == {val}--> {response.text}')

top_k == 1--> Okay, imagine you love yummy toast for breakfast, or delicious pasta for dinner, or even a cookie! All those things are made from something called **wheat**.

1.  **The Farmer's Job:** A farmer is like a superhero who grows the wheat in big fields. It's a very important job!
2.  **Sometimes it's Tricky:** But sometimes, growing wheat can be hard. Maybe it doesn't rain enough, or too much, or bugs eat some, or it just costs a lot of money to plant and harvest. If it's too hard, the farmer might say, "Oh dear, I can't grow wheat anymore!"
3.  **The Government Helps:** Now, there are grown-ups called the "government." They're like the big helpers for everyone in our country. They want to make sure *everyone* always has enough yummy bread and pasta to eat, and that it's not too, too expensive for mommies and daddies to buy.
4.  **The Special Money (Subsidy!):** So, the government gives the farmers a little extra money – like a special "helper bonus" – to make sure they can ke

#### Exercise 4

The same exercise as before but now with `top_p`, which controls how the model selects tokens for output `top_p=0.1` means the model selects tokens that make up 10% of the cumulative probability mass `top_p=0.9` means the model selects tokens that make up 90% of the cumulative probability mass `top_p` filters tokens *after* applying `top_k`.

Can you determine a rule of thumb as to how `top_k` and `top_p` affect the output results? (If you can't try to push the values to extreme values)

In [80]:
# Your code here
top_p_vals = [0.1,0.3,0.5,0.7,0.9]

for values in top_p_vals:
    config = types.GenerateContentConfig(
            temperature=0.4,
            top_p=values,
            top_k=20
    )
    response = client.models.generate_content(
                model=default_model,
                contents="Explain to me like I am 5: Why would a government grant subsidy to wheat farmers?",
                config=config
                )
    print(f'top_p == {values} --> {response.text}')

top_p == 0.1 --> Okay, imagine the grown-ups who help run our whole country are like the super-helpers for everyone! That's the **government**.

Now, imagine the people who plant tiny seeds and grow big fields of wheat. That's the **wheat farmers**. Wheat is super important because it makes flour, and flour makes yummy bread, cookies, and cereal!

Sometimes, it's really hard for the farmers. Maybe the weather is bad, or it costs a lot of money to buy seeds and big tractors. If it's too hard, they might stop growing wheat.

So, a **subsidy** is like the government giving the farmers a little extra money, like a special helper allowance!

Why do they do this?

1.  **So we always have enough food!** If farmers stop growing wheat, we wouldn't have enough bread or cereal to eat. The government wants to make sure everyone has yummy food on their table.
2.  **To make sure food isn't too expensive!** If there's not enough wheat, bread might cost a super, super lot of money. The government help

### 3. Add images to the prompt

#### Exercise 5
Gemini, beside text also accepts images (and videos). Try prompting it with one. Choose an interesting image and prompt the model with a query about it.

You can use the [official documentation](https://ai.google.dev/gemini-api/docs/vision?lang=python#prompting-images).

Use [PIL](https://pillow.readthedocs.io/en/stable/) to load an image. It should already be present in the Python environment.

In [81]:
from PIL import Image

IMAGE_PATH = "./data/engineer_fitting_prosthetic_arm.jpg"

# Your code here
image = Image.open(IMAGE_PATH)
text = "Give a description of what you see in the provided image."

response = client.models.generate_content(
    model=default_model,
    contents=[text, image, "Caption this image."],
    config=types.GenerateContentConfig(
            response_mime_type="application/json"
            )
)
print(response.text)

{
  "description": "Two men are in what appears to be an office or lab environment. On the right, a man with bilateral leg prostheses and a prosthetic left arm is seated in a wheelchair. He is wearing a blue shirt and khaki shorts. Another man, wearing a black t-shirt and dark pants, is standing or sitting at a tall table to the left. He is assisting the man in the wheelchair by adjusting or examining a part of his left arm prosthetic, specifically near the elbow. The man on the left also appears to be wearing a specialized glove or prosthetic on his left hand. A laptop is visible on the table. Large windows with blinds are in the background, providing natural light to the bright room."
}


### 4. Retrieval Augmented Generation (RAG)

#### Exercise 6

Depending on the application of the project, you might need to extract text from given documents and include it as additional context. This becomes especially relevant if you have many documents that cannot possibly fit into the model's context window. To more easily implement a RAG pipeline we recommend the use of one of these libraries: [LangChain](https://python.langchain.com/v0.2/docs/introduction/), [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/), [Haystack](https://docs.haystack.deepset.ai/docs/intro).

For the solution of this lab we will use *LangChain*.

It can be useful to split this exercise into these steps:
1. Read one or more documents using pdfminer
2. Split the documents into small chunks
3. Get and store the embeddings for each chunks
5. Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM

**NOTE:** if you try to embed too many documents at once or too large documents you may run into rate limits. Possible solutions: 
* Reduce the number of chunks and/or their size
* Look at the HF version of this lab and use a local embedding model

In [8]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.4 MB/s eta 0:00:000:00:01m eta 0:00:01


In [82]:
load_dotenv()

!if [ -z $GOOGLE_API_KEY ]; then echo "\$GOOGLE_API_KEY not found"; else echo "\$GOOGLE_API_KEY found"; fi

$GOOGLE_API_KEY found


I0000 00:00:1758266323.115763 5965093 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [91]:
import os  # langchain expects gemini's api key to be in the environment variable GOOGLE_API_KEY, use os to set it
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # get embeddings from Gemini
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
from pdfminer.high_level import extract_text  # extract text from pdfs
from uuid import uuid4
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain_google_genai import ChatGoogleGenerativeAI

DOC_PATH = "./data/chain_of_thought_prompting.pdf"

# Suppose a user query
USER_QUERY = "What is CoT?"

# Your code here
# 1. Read one or more documents using pdfminer
text = extract_text(DOC_PATH)
# text2 = extract_text('data/RP.pdf')

# 2. Split the documents into small chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=32,
    chunk_overlap=5,
    length_function=len,
    is_separator_regex=False
)
chunks_collection = text_splitter.split_text(text)

# A smaller version of the collection of text chunks (Gemini has API Rate limits)
small_cc = chunks_collection[:40]

# 3. Get and store the embeddings for each chunks
embedder = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
embeddings = embedder.embed_documents(small_cc)

# 4. Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM
vector_store = FAISS(
    embedding_function=embedder,
    index=faiss.IndexFlatL2(3072),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat4' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat7' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat4' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat7' is an invalid float value
Cannot set gray non-stroke color because /'pgfpat3' is an invali

In [92]:
# Convert the list of string chunks into Document() objects, and create a list of that
chunks_in_docs = [Document(page_content=chunk) for chunk in small_cc] # chunks in Documents form

uuids = [str(uuid4()) for _ in range(len(chunks_in_docs))] # make a randomly generated uuid, the number of times as length of docs
vector_store.add_documents(documents=chunks_in_docs, ids=uuids) # add them to the vector store to later be accessed

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [93]:
new_query = "What is CoT?" # What is CoT?
USER_QUERY = new_query if USER_QUERY != new_query else "What is CoT?"

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.4)

docs = vector_store.similarity_search(
    USER_QUERY,
    k=5
)

from langchain_core.messages import HumanMessage, SystemMessage

prompt_template = """
You are a helpful assistant that retrieves important information depending on the query asked, to solve a question asked by the user. You have information to help you to answer a question. You must not hallucinate. 

The information you know is: 
{context}
"""

similar_chunks = "".join(doc.page_content for doc in docs)
sys_prompt = prompt_template.format(context=similar_chunks)

question = llm.invoke([
    SystemMessage(content=sys_prompt),
    HumanMessage(content=USER_QUERY)
])
question

E0000 00:00:1758266935.794957 5965093 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


AIMessage(content='CoT stands for **Chain-of-Thought**.\n\nIt is a prompting technique used with large language models (LLMs) to improve their ability to solve complex reasoning tasks. This technique involves prompting the LLM to generate a series of intermediate reasoning steps, or a "chain of thought," before arriving at the final answer.\n\nThis approach mimics human-like problem-solving, where complex problems are broken down into smaller, more manageable steps. By explicitly generating these steps, CoT prompting helps LLMs to perform better on tasks that require multi-step reasoning, arithmetic, or symbolic manipulation. It has been shown to be particularly effective for tasks that are challenging for LLMs when only given a direct prompt.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1cbe141d-95b0-46b7-a61a-66c040017e93-0', usage_met

### 5. Explore on your own
Gemini offers a bigger range of capabilities than those provided here, begin able to automatically handle multi-turn chats is one of them. Explore them on your own!

#### Exercise 7
Explore!

In [ ]:
# Your code here

### 6. Create a user interface

#### Exercise 8
Since you are trying to build a complete application, you also need a nice user interface that interacts with the model. There are various libraries available for this purpose. Notably: [gradio](https://www.gradio.app/docs/gradio/interface) and [chat UI](https://huggingface.co/docs/chat-ui/index). For the solution of this lab, we will use gradio.

Gradio has pre-defined input/output blocks that are automatically inserted in the interface. You only need to provide an appropriate function that takes all the inputs and returns the relevant output. See documentation [here](https://www.gradio.app/docs/gradio/interface).

Use a ChatInterface to create a chatbot UI that let's you discuss with Gemini, then add multimodal capabilities for both Gradio and Gemini.

In [ ]:
import gradio as gr

# This part closes the demo server if it is already running (which
# happens easily in notebooks) and prevents you from opening multiple
# servers at the same time.
if "demo" in locals() and demo.is_running:
    demo.close()

# Edit the parameters below
chats = {}  # store the chat history for each user (suppose multiple users)

# Your code here